In [1]:
import pandas as pd
from pathlib import Path

PATH = Path.cwd().parent.joinpath('data')

# Goal

The goal of this notebook is to explore if we can use commodity trader's data for our analysis of the shadowfleet. Do they use vessels of the shadowfleet and how have they responded to the war in Ukraine? 

In [2]:
dfs = []
for file in PATH.joinpath('bloomberg', 'charters').glob('*.xlsx'):

    file_extension = file.suffix.lower()[1:]

    if file_extension == 'xlsx':
        df = pd.read_excel(file, engine='openpyxl', skiprows=1)
    elif file_extension == 'xls':
        df = pd.read_excel(file, skiprows=1)
    
    dfs.append(df)

fixtures = pd.concat(dfs)
fixtures.columns = fixtures.columns.str.lower()
fixtures.charterer = 'Vitol'

fixtures.dropna(subset=['imo_number'], inplace=True)
fixtures.drop_duplicates(inplace=True)
fixtures.reset_index(drop=True, inplace=True)
fixtures[['fixture_date', 'loading_date']] = fixtures[['fixture_date', 'loading_date']].apply(pd.to_datetime, errors='coerce')
fixtures.to_csv(PATH.joinpath('bloomberg', 'fixtures.csv'), index=False)

len(fixtures)

15102

In [16]:
uninsured = pd.read_csv(PATH.joinpath('processed', 'uninsured.csv'))
uninsured.start_date = pd.to_datetime(uninsured.start_date, errors='coerce')
uninsured.end_date = pd.to_datetime(uninsured.end_date, errors='coerce')
uninsured.earliest_sanction_date = pd.to_datetime(uninsured.earliest_sanction_date, errors='coerce')
uninsured.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 739 entries, 0 to 738
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   imo                     739 non-null    int64         
 1   start_date              739 non-null    datetime64[ns]
 2   end_date                739 non-null    datetime64[ns]
 3   earliest_sanction_date  84 non-null     datetime64[ns]
dtypes: datetime64[ns](3), int64(1)
memory usage: 23.2 KB


In [22]:
temp = pd.merge(fixtures, 
                uninsured, 
                left_on=['imo_number'],
                right_on='imo',
                how='left')
temp = temp[(temp.imo.notna()) & (temp.fixture_date >= temp.start_date) & (temp.fixture_date <= temp.end_date)].copy().reset_index(drop=True)
len(temp)

5

In [24]:
# How many vessels are in both datasets?

intersection = set(fixtures[fixtures.fixture_date > '2023-01-01'].imo_number).intersection(kse.imo)
len(intersection)

36

In [25]:
fixtures[fixtures.imo_number.isin(list(intersection))].imo_number.nunique()

36

## Berth visits

In [26]:
dfs = []
for file in PATH.joinpath('bloomberg', 'berth_visits').glob('*.xlsx'):

    file_extension = file.suffix.lower()[1:]

    if file_extension == 'xlsx':
        df = pd.read_excel(file, engine='openpyxl', skiprows=1)
    elif file_extension == 'xls':
        df = pd.read_excel(file, skiprows=1)
    
    dfs.append(df)

visits = pd.concat(dfs)


In [29]:
visits.drop_duplicates(inplace=True)
visits.columns = visits.columns.str.lower()

In [51]:
visits.to_csv(PATH.joinpath('bloomberg', 'berth_visits.csv'), index=False)

In [36]:
len(set(visits.vessel_imo).intersection(uninsured.imo))

316

In [41]:
visits.arrival_time = pd.to_datetime(visits.arrival_time, errors='coerce')

In [42]:
temp = pd.merge(visits,
                uninsured,
                left_on=['vessel_imo'],
                right_on='imo',
                how='left')

temp = temp[(temp.imo.notna()) & (temp.arrival_time >= temp.start_date) & (temp.arrival_time <= temp.end_date)].copy().reset_index(drop=True)

In [43]:
len(temp)

2312

In [45]:
temp.drop_duplicates(inplace=True)
len(temp)

1753

In [50]:
temp[temp.earliest_sanction_date.notna() & (temp.arrival_time >= temp.earliest_sanction_date)]

,berth_type,berth_name,port_country,vessel_class,estimated_quantity,vessel_imo,arrival_time,departure_time,vessel_destination_on_departure,vessel_name,maps_der,vessel_capacity,eta_for_next_destination_change,imo,start_date,end_date,earliest_sanction_date
768,Exports,Ust-Luga crude,Russia,Tanker-Aframax,118845,9258868,2024-02-27,2024-02-28 00:00:00,Ust-Luga,ASHER,IMO 9258868,121270,2024-02-27 00:00:00,9258868.0,2023-03-01,2024-06-30,2024-01-18
791,Exports,Ust-Luga crude,Russia,Tanker-Aframax,121610,9274800,2024-02-21,2024-02-23 00:00:00,Singapore,YANGTZE,IMO 9274800,124092,2024-03-09 00:00:00,9274800.0,2023-09-01,2024-04-30,2024-01-18
1347,Exports,Ust-Luga crude,Russia,Tanker-Aframax,118845,9258868,2024-06-20,2024-06-21 00:00:00,UST LUGA0,ASHER,IMO 9258868,121270,2024-09-07 00:00:00,9258868.0,2023-03-01,2024-06-30,2024-01-18
1569,Exports,Ust-Luga crude,Russia,Tanker-Aframax,121610,9274800,2024-04-26,2024-04-28 00:00:00,Ust-Luga,YANGTZE,IMO 9274800,124092,2024-05-25 00:00:00,9274800.0,2023-09-01,2024-04-30,2024-01-18
